In [0]:
layer = "gold"
nm_gold_staff = "kpi_staff"


In [0]:
%run ../functions

In [0]:
%sql
WITH limites AS (
  SELECT
    percentile_approx(resp_time, 0.15) AS p15,
    percentile_approx(resp_time, 0.85) AS p85
  FROM osticket.gold.fact_thread_ticket_agg
)
SELECT
  AVG(resp_time) AS avg_resp_time_sem_outliers
FROM osticket.gold.fact_thread_ticket_agg f
CROSS JOIN limites l
WHERE f.resp_time BETWEEN l.p15 AND l.p85;

In [0]:
%sql
with percent as (
  select
    percentile_approx(resp_time, 0.25) as q1,
    percentile_approx(resp_time, 0.75) as q3,
    percentile_approx(resp_time, 0.75)
      + 1.5 * (
        percentile_approx(resp_time, 0.75)
        - percentile_approx(resp_time, 0.25)
      ) as percent_limit
  from osticket.silver.thread_entry_agg
),

filtered_base as (
  select
    t.id_staff,
    t.resp_time,
    year(created) as year,
    month(created) as month,
    (year(created) * 100 + month(created)) as year_month
  from osticket.silver.thread_entry_agg t
  cross join percent p
  where t.resp_time <= p.percent_limit
  and t.id_staff is not null
)

select
  year_month,
  id_staff,
  year,
  month,
  median(resp_time) as median_time,
  floor(median_time/60) as hours,
  floor(mod(median_time, 60)) as minutes,
  concat(
  lpad(cast(hours as string), 2, '0'),
  ':',
  lpad(cast(minutes as string), 2, '0')
) as hours_minutes
from filtered_base
group by id_staff, year, month, year_month
order by id_staff, year, month, year_month

Decisão de uso da mediana e IQR:
-
-
Aqui foi decidio o uso de mediana ao uso de média, como o tempo de resposta em alguns chamados pode alterar de forma muito significativa por n motivos, assim pegaremos um valor central do tempo de resposta. Pra finalizar e dar um contexto mais real do tempo de resposta de cada staff, resolvi usar o método IQR para remover da contagem valores muito descrepantes, mesmo que a mediana já desse uma valor central, removendo esses outliers temos um valor mais acertivo para o nosso contexto.


In [0]:
df_staff = spark.sql("""
with percent as (
  select
    percentile_approx(resp_time, 0.25) as q1,
    percentile_approx(resp_time, 0.75) as q3,
    percentile_approx(resp_time, 0.75)
      + 1.5 * (
        percentile_approx(resp_time, 0.75)
        - percentile_approx(resp_time, 0.25)
      ) as percent_limit
  from osticket.silver.thread_entry_agg
),

filtered_base as (
  select
    t.id_staff,
    t.resp_time,
    year(created) as year,
    month(created) as month,
    (year(created) * 100 + month(created)) as year_month
  from osticket.silver.thread_entry_agg t
  cross join percent p
  where t.resp_time <= p.percent_limit
  and t.id_staff is not null
)

select
  year_month,
  id_staff,
  year,
  month,
  median(resp_time) as median_time,
  floor(median_time/60) as hours,
  floor(mod(median_time, 60)) as minutes,
  concat(
  lpad(cast(hours as string), 2, '0'),
  ':',
  lpad(cast(minutes as string), 2, '0')
) as hours_minutes
from filtered_base
group by id_staff, year, month, year_month
order by id_staff, year, month, year_month
""")
df_staff.display()


In [0]:
write_table(df_staff, layer, nm_gold_staff)